In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
foo = './drive/MyDrive/rscf/rusidiolect_data/processed_spacy/'
kws = pd.read_csv(foo+'kws_fpi_big5.csv', index_col = 0)
kws

,Min
feature,
я_PRON,133
свой_DET,76
они_PRON,66
он_PRON,58
хотеть_VERB,57
...,...
душа_NOUN,7
счастливый_ADJ,7
плохой_ADJ,6


In [ ]:
kwords = [x.split('_') for x in kws[kws.Min > 25].index]
kwords

[['я', 'PRON'],
 ['свой', 'DET'],
 ['они', 'PRON'],
 ['он', 'PRON'],
 ['хотеть', 'VERB'],
 ['время', 'NOUN'],
 ['человек', 'NOUN'],
 ['очень', 'ADV'],
 ['она', 'PRON'],
 ['думать', 'VERB'],
 ['мочь', 'VERB'],
 ['мой', 'DET'],
 ['мы', 'PRON'],
 ['жизнь', 'NOUN'],
 ['потом', 'ADV'],
 ['знать', 'VERB'],
 ['первый', 'ADJ'],
 ['год', 'NOUN'],
 ['хороший', 'ADJ'],
 ['еще', 'ADV'],
 ['произойти', 'VERB'],
 ['ждать', 'VERB'],
 ['день', 'NOUN'],
 ['видеть', 'VERB'],
 ['жить', 'VERB'],
 ['ничто', 'PRON']]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def getlinearlexcontexts(df, words, window = [-5, 5], 
                         lemfilter = [], posfilter = ['PUNCT'], sentborder = False):
  res, counts = {}, {}
  for i, (w, POS) in enumerate(words):
    name = w +'_' + POS
    occs = df[(df.Lemma == w) & (df.POS == POS)]
    counts[name] = occs.shape[0]
    def getborders(idx, sentborder = sentborder):
      if sentborder:
        punct = df[df.Lemma.isin(['.', '!', '?', '...'])]
        leftb = max([0, idx + window[0], punct[punct.index < idx].index.max()+1])
        #print([idx + window[1], len(df), punct[punct.index > idx].index.min()])
        rightb = min([idx + window[1]+1, len(df), punct[punct.index > idx].index.min()])
      else:
        leftb = max([0, idx + window[0]])
        rightb = min([idx + window[1]+1, len(df)])
      return [*range(leftb, idx), *range(idx+1, rightb)]
    #lb, rb = getborders(sentborder)
    windowsidx = [getborders(idx, sentborder) for idx in occs.index]
    #print(windowsidx)
    contdf = pd.DataFrame(columns = df.columns)
    if windowsidx:
      contdf = pd.concat([df.iloc[x] for x in windowsidx])
    contdf = contdf[~contdf.Lemma.isin(lemfilter) & ~contdf.POS.isin(posfilter)]
    res[name] = contdf[['Lemma', 'POS']].value_counts()
    res[name].index = ['_'.join(col).strip() for col in res[name].index.values] 
    res[name]['count'] = counts[name]
    #print(contdf.Lemma)
  return res
  #np.asarray(windowsidx).flatten()

In [ ]:
def getconttable(foo, words = [('он', 'PRON'), ('я', 'PRON')], 
                 resfn = None,
                 resfoo = None):
  resd = {}
  for i, fn in enumerate(os.listdir(foo)):
    #print(fn)
    df = pd.read_csv(foo+fn)
    df.rename(mapper = {'Unnamed: 0':'Position'}, axis = 1, inplace = True)
    conts = getlinearlexcontexts(df, words = words)
    if resfoo:
      resd[fn[:-4]] = conts
    else:
      resd[fn[:-4]] = pd.concat(conts)
    if i%50 == 0:
      print(fn, i, 'ready.')
  if resfoo:
    for w in resd[fn[:-4]].keys():
      #print(resd.keys())
      res = pd.DataFrame().from_dict({fna : resd[fna][w] for fna in resd.keys()}, 
                                     orient = 'index')
      res.fillna(0).to_csv(resfoo + w + '.tsv', sep = '\t')
  else:
    res = pd.DataFrame().from_dict(resd, orient = 'index')
    res.fillna(0.0, inplace = True)
    res.to_csv(resfn, sep = '\t')
  return res

In [ ]:
for fo in ['big5_536/', 'fpi_1045/']:
  print(fo)
  res = getconttable(foo = foo + fo, 
                   words = kwords, 
                   resfoo = foo + 'lex_contexts/' + fo)

big5_536/
1RNPВ_25_м_0_описание.csv 0 ready.
26RNPВ_22_ж_1_описание.csv 50 ready.
51RNPВ_20_ж_1_описание.csv 100 ready.
78RNPВ_23_ж_0_повествование.csv 150 ready.
104RNPВ_24_ж_1_описание.csv 200 ready.
134RNPВ_19_ж_0_повествование.csv 250 ready.
160RNPВ_19_м_1_описание.csv 300 ready.
188RNPВ_19_м_0_повествование.csv 350 ready.
6RP1_23_ж_0_описание.csv 400 ready.
32RP1_20_м_0_описание.csv 450 ready.
58RP1_20_м_0_описание.csv 500 ready.
fpi_1045/
34RP2_nan_ж_1_официальный.csv 0 ready.
60RP2_nan_ж_0_повествование.csv 50 ready.
92RP2_nan_ж_0_повествование.csv 100 ready.
120RP2_nan_ж_1_повествование.csv 150 ready.
146RP2_nan_м_0_описание.csv 200 ready.
171RP2_nan_м_0_описание.csv 250 ready.
196RP2_nan_м_0_повествование.csv 300 ready.
222RP2_nan_ж_0_повествование.csv 350 ready.
248RP2_nan_ж_1_описание.csv 400 ready.
273RP2_nan_ж_1_описание.csv 450 ready.
299RP2_nan_ж_0_повествование.csv 500 ready.
324RP2_nan_ж_1_официальный.csv 550 ready.
349RP2_nan_ж_1_официальный.csv 600 ready.
377RP2_n

In [ ]:
%%time
res1 = getconttable(foo = './drive/MyDrive/rscf/rusidiolect_data/processed_spacy/fpi_1045/', 
                    words = [('он', 'PRON'), ('я', 'PRON')], 
                   resfoo = './drive/MyDrive/rscf/rusidiolect_data/processed_spacy/lex_contexts/fpi_1045/')

34_nan_ж_1_официальный.csv 0 ready.
60_nan_ж_0_повествование.csv 50 ready.
92_nan_ж_0_повествование.csv 100 ready.
120_nan_ж_1_повествование.csv 150 ready.
146_nan_м_0_описание.csv 200 ready.
171_nan_м_0_описание.csv 250 ready.
196_nan_м_0_повествование.csv 300 ready.
222_nan_ж_0_повествование.csv 350 ready.
248_nan_ж_1_описание.csv 400 ready.
273_nan_ж_1_описание.csv 450 ready.
299_nan_ж_0_повествование.csv 500 ready.
324_nan_ж_1_официальный.csv 550 ready.
349_nan_ж_1_официальный.csv 600 ready.
377_nan_ж_0_повествование.csv 650 ready.
402_nan_ж_0_повествование.csv 700 ready.
429_nan_ж_1_официальный.csv 750 ready.
455_nan_ж_1_официальный.csv 800 ready.
481_nan_ж_0_повествование.csv 850 ready.
507_nan_ж_0_повествование.csv 900 ready.
532_nan_ж_0_повествование.csv 950 ready.
1_nan_ж_0_повествование.csv 1000 ready.
CPU times: user 32.9 s, sys: 1.11 s, total: 34.1 s
Wall time: 3min 2s


In [ ]:
585*6

3510